In [81]:
import pandas as pd, numpy as np
import json

In [82]:
progress = pd.read_csv('historical_progresses_jan.csv')

In [83]:
progress[progress['subjectCode'] == 'en'].groupby(['subjectCode', 'studentName', 'lessonId']).agg({'score': [np.max], 'lessonOrder': lambda x: np.mean(x)})

score lessonOrder
                                    amax    <lambda>
subjectCode studentName lessonId                    
en          Aadharsha   balloon25     60       149.0
                        balloon61     70       298.0
                        en0000       100         1.0
                        en0001        72         2.0
                        en0500        64        59.0
...                                  ...         ...
            vikasha     balloon6      20        62.0
                        en0500        83        59.0
                        en0501        69        60.0
                        en0502        50        61.0
                        en_PreQuiz    42         0.0

[9600 rows x 2 columns]

In [84]:
students = {}
for row in progress.itertuples():
    score = row[7]
    student = row[10]
    subject = row[12]
    lesson_order = row[14]
    date_time = row[13]
    if(student == 'nan' or student == '2'):
        print(row)
    if student not in students:
        students[student] = {subject: {lesson_order: (score, date_time)}}
    elif subject not in students[student]:
        students[student][subject] = {lesson_order: (score, date_time)}
    elif lesson_order not in students[student][subject]:
        students[student][subject][lesson_order] = (score, date_time)
    elif students[student][subject][lesson_order][0] < score:
        students[student][subject][lesson_order] = (score, date_time)


Pandas(Index=29427, id='Pr8afOCRmIT16XSri8y2', progressId='ff1ef8a1-5b25-4727-b11a-304ae688fcb8', chapterId='kn20', chapterName='ಗುಣಿತಾಕ್ಷರಗಳು -3', lessonId='kn2010', lessonName='ಪದ ಕಲಿಯಿರಿ   -  5', score=93, schoolId='QwdOto9bdsSGLwf6G7Kd', sectionId='hh7Wczfy3yXOeK9kBiQI', studentId='2', studentName='Kushi', subjectCode='kn', date='Mon Jan 03 2022 14:36:28 GMT+0530 (India Standard Time)', lessonOrder=158)
Pandas(Index=29428, id='L11gy2sW4ERkAruyaY5G', progressId='ff1ef8a1-5b25-4727-b11a-304ae688fcb8', chapterId='kn20', chapterName='ಗುಣಿತಾಕ್ಷರಗಳು -3', lessonId='kn2011', lessonName='ಪದ ಕಲಿಯಿರಿ -  6', score=93, schoolId='QwdOto9bdsSGLwf6G7Kd', sectionId='hh7Wczfy3yXOeK9kBiQI', studentId='2', studentName='Kushi', subjectCode='kn', date='Mon Jan 03 2022 14:42:55 GMT+0530 (India Standard Time)', lessonOrder=159)
Pandas(Index=29429, id='TdoXoQWL0NxYnGPgXmxt', progressId='ff1ef8a1-5b25-4727-b11a-304ae688fcb8', chapterId='kn21', chapterName='ಅಭ್ಯಾಸ -3', lessonId='kn2100', lessonName='ಪದ ಕಲಿಯಿ

In [85]:
print(students.keys())

dict_keys(['KD07', 'KD10', 'KD01', 'KD08', 'KD09', 'KD06', 'KD05', 'KD02', 'KD03', 'KD04', 'KD11', 'KD12', 'CV01', 'CV05', 'CV08', 'CV09', 'CV02', 'CV04', 'CV03', nan, 'CV10', 'CV07', 'CV06', 'HA06', 'HA14', 'awsd', 'HA02', 'HA10', 'HA09', 'HA05', 'aq2', 'HA15', 'HA08', 'HA03', 'HA11', 'HA01', 'HA04', 'HA12', 'HA13', 'RA52', 'RA47', 'RA50', 'RA36', 'RA39', 'RA46', 'RA44', 'RA42', 'RA38', 'RA60', 'RA45', 'RA40', 'RA48', 'RA37', 'RA33', 'RA41', 'RA32', 'RA58', 'RA53', 'RA54', 'RA57', 'RA35', 'RA61', 'RA49', 'RA55', 'RA34', 'RA43', 'RA51', 'RA59', 'RA56', 'RA26', 'RA16', 'RA05', 'RA08', 'RA15', 'RA03', 'RA18', 'RA11', 'RA21', 'RA02', 'RA07', 'RA22', 'RA27', 'RA14', 'RA25', 'RA09', 'RA20', 'RA23', 'RA12', 'RA19', 'RA10', 'RA31', 'RA28', 'RA06', 'RA24', 'RA04', 'RA13', 'RA29', 'KU18', 'KU14', 'KU16', 'KU21', 'KU15', 'KU11', 'KU17', 'KU10', 'KU20', 'KU19', 'KU09', 'KU12', 'KU13', 'KD16', 'KD17', 'KU07', 'KU02', 'KU08', 'KD15', 'KU01', 'KU06', 'KU05', 'KU03', 'KU04', 'SH31', 'SH26', 'SH34', '

In [86]:
max_lessons = {
    'kn': 379/2,
    'en': 303/2,
    'maths': 404/2
}
# 1, 1.5, 2.5, 3, 4, 5, 6
month = 5
total_months = 10
alpha_avg = 0
for student, course in students.items():
    course_avg = {
        'kn': 0,
        'en': 0,
        'maths': 0
    }
    lesson_details = []
    for course_name, lessons in course.items():
        if course_name in max_lessons:
            lesson_avg = 0
            score_avg = 0
            for lesson_order, score in lessons.items():
                lesson_avg += lesson_order
                score_avg += score[0]
            lesson_avg = lesson_avg / max(max_lessons[course_name]/total_months, len(lessons))
            ideal_lesson_avg = max_lessons[course_name] * (month - 0.5)/total_months
            score_avg /= len(lessons)
            # print(course_name, lesson_avg, ideal_lesson_avg, lesson_avg/ideal_lesson_avg, score_avg, len(lessons), max_lessons[course_name]/total_months)
            course_avg[course_name] = lesson_avg/ideal_lesson_avg * (score_avg + 15) / 100
            lesson_details.append({'course': course_name, 'lesson_num': lesson_order, 'score': score[0], 'date': score[1]})
    alpha = (course_avg['kn'] + course_avg['en'] + course_avg['maths']) / 3
    print(student, '\t', alpha, '\t', json.dumps(lesson_details))
    alpha_avg += alpha
alpha_avg = alpha_avg / len(students)

KD07 	 0.08374088233905898 	 [{"course": "maths", "lesson_num": 84, "score": 96, "date": "Mon Jan 03 2022 11:27:33 GMT+0530 (India Standard Time)"}]
KD10 	 0.7355733878731913 	 [{"course": "kn", "lesson_num": 103, "score": 66, "date": "Mon Jan 03 2022 11:25:34 GMT+0530 (India Standard Time)"}, {"course": "en", "lesson_num": 4, "score": 88, "date": "Mon Jan 03 2022 12:17:06 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 16, "score": 100, "date": "Tue Jan 25 2022 12:05:57 GMT+0530 (India Standard Time)"}]
KD01 	 0.3460328577197942 	 [{"course": "en", "lesson_num": 48, "score": 73, "date": "Wed Jan 12 2022 14:47:03 GMT+0530 (India Standard Time)"}, {"course": "kn", "lesson_num": 10, "score": 88, "date": "Fri Dec 31 2021 12:38:38 GMT+0530 (India Standard Time)"}, {"course": "maths", "lesson_num": 17, "score": 46, "date": "Tue Jan 25 2022 11:58:31 GMT+0530 (India Standard Time)"}]
KD08 	 0.2351598517384719 	 [{"course": "maths", "lesson_num": 57, "score": 34, "date": "M

In [87]:
print(alpha_avg)

0.5796270093977828
